In [1]:
import pandas as pd

In [2]:
demo1 = pd.read_csv('../data/demo-1.csv')

In [3]:
demo1

,road,id,model_type,name,lat,lon,length
0,N1,1000000,source,source,0,0,4
1,N1,1000001,link,link 1,1,1,1800
2,N1,1000002,bridge,bridge 1,2,2,200
3,N1,1000003,link,link 2,3,3,1500
4,N1,1000004,bridge,bridge 2,4,4,8
5,N1,1000005,link,link 3,5,5,2000
6,N1,1000006,bridge,bridge 3,6,6,4
7,N1,1000007,link,link 4,7,7,2500
8,N1,1000008,bridge,bridge 4,8,8,500
9,N1,1000009,link,link 5,9,9,800


In [4]:
roads = pd.read_csv('../data/_roads3.csv')
bmms = pd.read_excel('../data/BMMS_overview.xlsx')

In [5]:
roads[roads["road"] == "N1"]

,road,chainage,lrp,lat,lon,gap,type,name
0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...
1,N1,0.814,LRPSa,23.702917,90.450417,NaN,Culvert,Box Culvert
2,N1,0.822,LRPSb,23.702778,90.450472,NaN,CrossRoad,Intersection with Z1101
3,N1,1.000,LRP001,23.702139,90.451972,NaN,KmPost,Km post missing
4,N1,2.000,LRP002,23.697889,90.460583,NaN,KmPost,Km post missing
...,...,...,...,...,...,...,...,...
1334,N1,461.476,LRP466a,20.868860,92.298222,NaN,Culvert,Box culvert
1335,N1,461.904,LRP466b,20.865028,92.298250,BS,Bridge,Bridge start
1336,N1,461.946,LRP466c,20.864667,92.298194,BE,Bridge,Bridge end
1337,N1,462.124,LRP467,20.862972,92.298083,NaN,KmPost,Infor.missing


In [6]:
bmms[bmms['road'] == 'N1']

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.800,19.5,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.698739,90.458861,interpolate
1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.4,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.694664,90.487775,interpolate
2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.705060,90.523214,interpolate
3,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.30,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.880,12.2,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.694391,90.537574,interpolate
4,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.30,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.2,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.694302,90.537707,interpolate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19378,N1,426.915,RCC Girder Bridge,LRP423b,Balu Khali Bridge,7.90,D,120493,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,426.915,NaN,NaN,NaN,Chittagong,Chittagong,Cox's Bazar,Cox's Bazar - 2,21.129768,92.187047,interpolate
19379,N1,427.622,RCC Girder Bridge,LRP424a,Wheke Kang Bridge,6.40,D,120498,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,427.622,NaN,NaN,NaN,Chittagong,Chittagong,Cox's Bazar,Cox's Bazar - 2,21.125104,92.191367,interpolate
19380,N1,429.081,RCC Girder Bridge,LRP425c,Tasse Bridge,12.80,D,120507,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,429.081,NaN,NaN,NaN,Chittagong,Chittagong,Cox's Bazar,Cox's Bazar - 2,21.114558,92.198021,interpolate
19381,N1,436.789,RCC Girder Bridge,LRP433b,Naya Bazar(2) Bridge,9.40,D,120524,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,436.789,NaN,NaN,NaN,Chittagong,Chittagong,Cox's Bazar,Cox's Bazar - 2,21.056294,92.226645,interpolate


In [7]:
# Define the columns for the empty DataFrame
columns = ['road', 'id', 'model_type', 'name', 'lat', 'lon', 'length']

# Create an empty DataFrame with the specified columns
df = pd.DataFrame(columns=columns)

# Display the empty DataFrame
df

,road,id,model_type,name,lat,lon,length


In [8]:
# Filter the 'roads' DataFrame for rows where the 'road' column is 'N1'
n1_roads = roads[roads['road'] == 'N1']

# Generate an ID sequence starting from 1
n1_roads['id'] = range(1, len(n1_roads) + 1)

# Set model_type to 'link' for all rows
n1_roads['model_type'] = 'link'

# Generate 'name' as "Link" + id as string
n1_roads['name'] = ['Link ' + str(id) for id in n1_roads['id']]

# Calculate 'length' as difference between this row's 'chainage' and the next row's 'chainage'
# Shift(-1) moves the chainage up by one row to subtract, fillna(0) to handle the last item
n1_roads['length'] = (n1_roads['chainage'].shift(-1) - n1_roads['chainage']).fillna(0)

# Selecting the columns needed for the empty DataFrame
n1_roads_final = n1_roads[['road', 'id', 'model_type', 'name', 'lat', 'lon', 'length', 'chainage']]

# Display the first few rows of the final DataFrame
n1_roads_final


C:\Users\svenk\AppData\Local\Temp\ipykernel_23680\2401421442.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n1_roads['id'] = range(1, len(n1_roads) + 1)
C:\Users\svenk\AppData\Local\Temp\ipykernel_23680\2401421442.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n1_roads['model_type'] = 'link'
C:\Users\svenk\AppData\Local\Temp\ipykernel_23680\2401421442.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,road,id,model_type,name,lat,lon,length,chainage
0,N1,1,link,Link 1,23.706028,90.443333,0.814,0.000
1,N1,2,link,Link 2,23.702917,90.450417,0.008,0.814
2,N1,3,link,Link 3,23.702778,90.450472,0.178,0.822
3,N1,4,link,Link 4,23.702139,90.451972,1.000,1.000
4,N1,5,link,Link 5,23.697889,90.460583,0.130,2.000
...,...,...,...,...,...,...,...,...
1334,N1,1335,link,Link 1335,20.868860,92.298222,0.428,461.476
1335,N1,1336,link,Link 1336,20.865028,92.298250,0.042,461.904
1336,N1,1337,link,Link 1337,20.864667,92.298194,0.178,461.946
1337,N1,1338,link,Link 1338,20.862972,92.298083,0.130,462.124


In [9]:
# Filter BMMS data for road 'N1'
bmms_n1 = bmms[bmms['road'] == 'N1'].copy()

# Set up for new entries
bmms_n1['model_type'] = 'bridge'
bmms_n1['name'] = ['Bridge ' + str(i+1) for i in range(bmms_n1.shape[0])]
bmms_n1['id'] = range(n1_roads_final['id'].max() + 1, n1_roads_final['id'].max() + 1 + bmms_n1.shape[0])
bmms_n1['chainage'] = bmms_n1['km']  # Use 'km' as 'chainage'
bmms_n1['length'] = bmms_n1['length'] / 1000

# Select and rename columns to match the format of `n1_roads_final_with_chainage`
bmms_n1_formatted = bmms_n1[['road', 'id', 'model_type', 'name', 'lat', 'lon', 'chainage', 'length', 'condition']]

# Combine the dataframes and sort by chainage
combined_df = pd.concat([n1_roads_final, bmms_n1_formatted], ignore_index=True).sort_values(by='chainage')

# Display the first few rows of the combined and sorted DataFrame
combined_df


,road,id,model_type,name,lat,lon,length,chainage,condition
0,N1,1,link,Link 1,23.706028,90.443333,0.8140,0.000,NaN
1,N1,2,link,Link 2,23.702917,90.450417,0.0080,0.814,NaN
2,N1,3,link,Link 3,23.702778,90.450472,0.1780,0.822,NaN
3,N1,4,link,Link 4,23.702139,90.451972,1.0000,1.000,NaN
1339,N1,1340,bridge,Bridge 1,23.698739,90.458861,0.0113,1.800,A
...,...,...,...,...,...,...,...,...,...
1334,N1,1335,link,Link 1335,20.868860,92.298222,0.4280,461.476,NaN
1335,N1,1336,link,Link 1336,20.865028,92.298250,0.0420,461.904,NaN
1336,N1,1337,link,Link 1337,20.864667,92.298194,0.1780,461.946,NaN
1337,N1,1338,link,Link 1338,20.862972,92.298083,0.1300,462.124,NaN


In [15]:
combined_df.iloc[0, 2] = 'source'
combined_df.iloc[-1, combined_df.columns.get_loc('model_type')] = 'sink'
combined_df.reset_index(drop=True, inplace=True)
count = 1
for index, row in combined_df.iterrows():
    combined_df.iloc[index, 1] = count
    count += 1


In [16]:
combined_df.to_csv('../data/n1_model.csv', index=False)

In [17]:
n1 = pd.read_csv('../data/n1_model.csv')

In [18]:
n1

,road,id,model_type,name,lat,lon,length,chainage,condition
0,N1,1,source,Link 1,23.706028,90.443333,0.8140,0.000,NaN
1,N1,2,link,Link 2,23.702917,90.450417,0.0080,0.814,NaN
2,N1,3,link,Link 3,23.702778,90.450472,0.1780,0.822,NaN
3,N1,4,link,Link 4,23.702139,90.451972,1.0000,1.000,NaN
4,N1,5,bridge,Bridge 1,23.698739,90.458861,0.0113,1.800,A
...,...,...,...,...,...,...,...,...,...
2121,N1,2122,link,Link 1335,20.868860,92.298222,0.4280,461.476,NaN
2122,N1,2123,link,Link 1336,20.865028,92.298250,0.0420,461.904,NaN
2123,N1,2124,link,Link 1337,20.864667,92.298194,0.1780,461.946,NaN
2124,N1,2125,link,Link 1338,20.862972,92.298083,0.1300,462.124,NaN
